In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [2]:
raw_df = pd.read_csv("rawdata.csv",encoding='ISO-8859-1')
var_df = pd.read_csv("Varlist.csv")

In [3]:
raw_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1211 entries, 0 to 1210
Columns: 166 entries, STARTDATE to DEMOGRAPHIC13
dtypes: bool(1), float64(2), int64(5), object(158)
memory usage: 1.5+ MB


### Figuring out what the types for each feature is

In [8]:
strings = raw_df[raw_df.select_dtypes(include=['object']).columns].columns.to_list() 
bools = raw_df[raw_df.select_dtypes(include=['bool']).columns].columns.to_list() 
print(bools)
print(strings)
print(len(strings))
intAndFloats = raw_df[raw_df.select_dtypes(include=['int64', 'float64']).columns].columns.to_list() 
print(intAndFloats)

[]
['STARTDATE', 'ENDDATE', 'FINISHED', 'RECORDEDDATE', 'DISTRIBUTIONCHANNEL', 'USERLANGUAGE', 'ARM', 'CONSENT', 'COMPREHENSION1_ARM1', 'COMPREHENSION1_ARM2', 'COMPREHENSION2_ARM1', 'COMPREHENSION2_ARM2', 'HR1', 'HR2_1', 'HR2_2', 'HR2_3', 'HR2_4', 'HR2_5', 'HR3', 'HR4', 'HR5', 'HR6', 'HR7_1', 'HR7_2', 'HR7_3', 'HR7_4', 'HR8', 'HR10', 'HR11', 'HR11_6_TEXT', 'HR12_1', 'HR12_2', 'HR12_3', 'HR12_4', 'HR12_5', 'HR13', 'HR14', 'HR14_10_TEXT', 'COVID_HR1', 'COVID_HR2', 'COVID_HR2_11_TEXT', 'COVID_HR3', 'STIMULUS1', 'STIMULUS2', 'STIMULUS3', 'STIMULUS4', 'STIMULUS5', 'STIMULUS5_8_TEXT', 'STIMULUS6', 'STIMULUS6_7_TEXT', 'POLICY1', 'POLICY2', 'POLICY3', 'POLICY4', 'POLICY5', 'POLICY5_13_TEXT', 'POLICY6', 'POLICY6_10_TEXT', 'PERSONAL1', 'PERSONAL2', 'PERSONAL3', 'PERSONAL4', 'PERSONAL5', 'PERSONAL6', 'PERSONAL6_13_TEXT', 'PERSONAL7', 'PERSONAL7_11_TEXT', 'WORRY_1', 'WORRY_2', 'WORRY_3', 'WORRY_4', 'WORRY_5', 'WORRY_6', 'WORRY_7', 'WORRY_8', 'MEDIA1', 'MEDIA2', 'MEDIA3', 'MEDIA3_11_TEXT', 'MEDIA4'

cont: Duration, 
ratio: demographics3, duration, elderlycare1_1
ordinal: HR1_10SCALE
nominal: arm, hr1, 

In [9]:
#Check 7 columns that contain int64 or float 64 data, 3 of them are categorical data, move to cat_feature array
#convert int columns to string columns because they are categorical data
raw_df['HR1_10SCALE'] = raw_df['HR1_10SCALE'].apply(str)
raw_df['ARM'] = raw_df['ARM'].apply(str)
raw_df['HR1'] = raw_df['HR1'].apply(str)
raw_df['FINISHED'] = raw_df['FINISHED'].apply(str)
cat_features = strings + bools + ['HR1_10SCALE', 'ARM', 'HR1'] 
cont_features = ["PROGRESS", "DURATION", "DEMOGRAPHIC3", "ELDERLYCARE1_1"] 

In [10]:
raw_df.shape 
totalRows=len(raw_df.index)
print(totalRows)

1211


### Summary tables

In [11]:
#Define Function for analyzing continuous variables
def subString (string):
    if isinstance(string, np.integer):
        return "{}".format(string)
    
    if type(string)==bool:
        return string
    
    if string is None:
        return ""
    
    if len(string) < 10:
        return string
    else:
        return string[:10]  + "..."
    
def cont_summary(feature_name, data): 
    summary_feature_names = ['Feature', 'Count', '% of Missing', 'Card.', 'Min.', 'Q1', 'Median', 
                            'Q3', 'Max.', 'Mean', 'Std. Dev.']
    # Your answer to Q1 goes here! Make sure to return a DataFrame with the features specified. 
    newDf = pd.DataFrame(columns=summary_feature_names)
    newDf.at[0,'Feature'] = feature_name
    newDf.at[0,'Card.'] = data[feature_name].nunique()
    newDf.at[0,'Count'] = data[feature_name].count()
    newDf.at[0,'% of Missing'] = '{:.2f}%'.format((data[feature_name].isnull().sum()/totalRows)*100)
    newDf.at[0,'Min.'] = data[feature_name].min()
    newDf.at[0,'Q1'] = data[feature_name].quantile(0.25)
    newDf.at[0,'Mean'] = '{:.2f}'.format(data[feature_name].mean())
    newDf.at[0,'Median'] = data[feature_name].quantile(0.50)
    newDf.at[0,'Q3'] = data[feature_name].quantile(.75)
    newDf.at[0,'Max.'] = data[feature_name].max()
    newDf.at[0,'Std. Dev.'] = '{:.2f}'.format(data[feature_name].std())
    IQR = data[feature_name].quantile(.75) - data[feature_name].quantile(0.25)
    Q1 = data[feature_name].quantile(0.25)
    Q3 = data[feature_name].quantile(.75)
    return newDf

#Define Function for analyzing categorical variables
def cat_summary(feature_name,data):
    summary_feature_names = ['Feature', 'Count', '% of Missing', 'Card.', 'Mode', 'Mode Freq.', 'Mode %', 
                            '2nd Mode', '2nd Mode Freq.', '2nd Mode Perc']
    if feature_name == 'HR1_10SCALE':
        data[feature_name] = data[feature_name].astype("string") 
    newDf = pd.DataFrame(columns=summary_feature_names)
    newDf.at[0,'Feature'] = feature_name
    newDf.at[0,'Card.'] = data[feature_name].nunique()
    newDf.at[0,'Count'] = data[feature_name].count() 
    newDf.at[0,'% of Missing'] =  '{:.2f}%'.format((data[feature_name].isnull().sum()/totalRows)*100)      
    newDf.at[0,'Mode'] = subString(data[feature_name].value_counts().index[0])     
    newDf.at[0,'Mode Freq.'] = data[feature_name].value_counts()[0]
    newDf.at[0,'Mode %'] = '{:.2f}%'.format((data[feature_name].value_counts()[0]/totalRows)*100)
    if (data[feature_name].value_counts()[0]/totalRows)*100 != 100: 
        newDf.at[0,'2nd Mode'] = subString(data[feature_name].value_counts().index[1])
        newDf.at[0,'2nd Mode Freq.'] = data[feature_name].value_counts()[1]
        newDf.at[0,'2nd Mode Perc'] = '{:.2f}%'.format((data[feature_name].value_counts()[1]/totalRows)*100)
    return newDf

In [12]:
#analyzing continuous variables
summary_feature_names = ['Feature',  'Count', '% of Missing', 'Card.', 'Min.', 'Q1', 'Median','Q3', 'Max.', 'Mean', 'Std. Dev.']
quality_cont = pd.DataFrame(columns=summary_feature_names) 
for i in cont_features:
    tmp = cont_summary(i,raw_df)
    quality_cont = quality_cont.append(tmp.loc[0], ignore_index=True) 
print(quality_cont) 
quality_cont.to_csv("cont_summary.csv")

#analyzing categorical variables
summary_feature_names = ['Feature',  'Count', '% of Missing', 'Card.', 'Mode', 'Mode Freq.', 'Mode %','2nd Mode', '2nd Mode Freq.', '2nd Mode Perc']
quality_cat = pd.DataFrame(columns=summary_feature_names)
for i in cat_features:
#for i in strings:
    #if i!= "HR1" or i != 'FINISHED':
    tmp = cat_summary(i,raw_df)
    quality_cat = quality_cat.append(tmp.loc[0], ignore_index=True)
print(quality_cat)

quality_cat.to_csv("cat_summary.csv")

          Feature Count % of Missing Card.  Min.      Q1  Median      Q3  \
0        PROGRESS  1211        0.00%     1   100   100.0   100.0   100.0   
1        DURATION  1211        0.00%   983   232   717.5  1168.0  1881.5   
2    DEMOGRAPHIC3  1211        0.00%    80  1924  1971.0  1982.0  1990.0   
3  ELDERLYCARE1_1   331       72.67%    30   0.0     1.0     1.0     2.0   

           Max.         Mean     Std. Dev.  
0           100       100.00          0.00  
1         45050      1698.93       2354.99  
2          2010      1979.17         15.84  
3  9867436378.0  29811439.38  542363329.34  
                 Feature Count % of Missing Card.           Mode Mode Freq.  \
0              STARTDATE  1211        0.00%   590  6/3/2020 1...         19   
1                ENDDATE  1211        0.00%   735  6/3/2020 1...          9   
2               FINISHED  1211        0.00%     1           True       1211   
3           RECORDEDDATE  1211        0.00%   733  6/3/2020 1...          9   

Our main features are these. Of course they are broken into multiple groups for each but this is mainly what we are working with.
'STARTDATE', 'ENDDATE', 'PROGRESS', 'DURATION', 'FINISHED', 'RECORDEDDATE', 'DISTRIBUTIONCHANNEL', 'USERLANGUAGE', 'ARM', 'CONSENT', 'COMPREHENSION1_ARM1', 'HR1',  'COVID_HR1', 'STIMULUS1', 'POLICY1', 'PERSONAL1', 'WORRY_1', 'MEDIA1',  'MH1', 'CHILDCARE1','ELDERLYCARE1', 'IDEOLOGY1', 'CULTURE1_1', 'RUMORS1','DEMOGRAPHIC1'

In [13]:
#Check for unique value counts, if counts are too low, there is no statistics values. 
single_value_column = []
multi_value_card_cat = []
for index, row in quality_cat.iterrows():
    if quality_cat['Card.'][index] == 1:
        single_value_column.append(quality_cat['Feature'][index])
    else:
        multi_value_card_cat.append(quality_cat['Feature'][index])
print(len(single_value_column))
print(len(multi_value_card_cat))

6
158


### Check Missing Values

In [14]:
#drop STARTDATE ENDDATE FINISHED	RECORDEDDATE	DISTRIBUTIONCHANNEL
#For all cat_features, check data lenght to find missing data
too_many_missing_value_columns = []
totalRows = len(raw_df.index)
for (columnName, columnData) in raw_df.iteritems(): 
    missingCount = columnData.isnull().sum() 
    percentage = (missingCount/totalRows)*100 
    #print(percentage)
    if missingCount > 0:        
        print("Column: ", columnName.strip(), ", missing rows: ", 
            missingCount, ", missing percentage: ",  '{:.2f}%'.format(percentage)) 
        if percentage > 50.0:
            too_many_missing_value_columns.append(columnName) 

Column:  COMPREHENSION1_ARM1 , missing rows:  829 , missing percentage:  68.46%
Column:  COMPREHENSION1_ARM2 , missing rows:  829 , missing percentage:  68.46%
Column:  COMPREHENSION2_ARM1 , missing rows:  820 , missing percentage:  67.71%
Column:  COMPREHENSION2_ARM2 , missing rows:  820 , missing percentage:  67.71%
Column:  HR10 , missing rows:  913 , missing percentage:  75.39%
Column:  HR11 , missing rows:  2 , missing percentage:  0.17%
Column:  HR11_6_TEXT , missing rows:  1177 , missing percentage:  97.19%
Column:  HR14_10_TEXT , missing rows:  1203 , missing percentage:  99.34%
Column:  COVID_HR1 , missing rows:  485 , missing percentage:  40.05%
Column:  COVID_HR2 , missing rows:  404 , missing percentage:  33.36%
Column:  COVID_HR2_11_TEXT , missing rows:  1200 , missing percentage:  99.09%
Column:  STIMULUS5_8_TEXT , missing rows:  1193 , missing percentage:  98.51%
Column:  STIMULUS6_7_TEXT , missing rows:  1166 , missing percentage:  96.28%
Column:  POLICY5_13_TEXT , miss

In [15]:
#Step 1: drop columns with single value or more than 50% missing data
for col1 in single_value_column:
    if col1 in raw_df.columns:
        raw_df.drop(col1, axis = 1, inplace=True)
    
for col2 in too_many_missing_value_columns:
    if col2 in raw_df.columns:
        raw_df.drop(col2, axis = 1, inplace=True)

#Column numbers are down from 166 to 125
raw_df.shape 

(1211, 124)

In [16]:
#Step 2: drop survey STARTDATE, ENDDATE, PROGRESS, DURATION, FINISHED, RECORDEDDATE 
columnList = ['STARTDATE', 'ENDDATE', 'PROGRESS', 'DURATION', 'FINISHED', 'RECORDEDDATE']
for column in columnList:
    if  column in raw_df.columns:
        raw_df.drop(column, axis = 1, inplace=True)
raw_df.shape 

(1211, 119)

In [17]:
#Step 3: cacluate age by giving birth year 
if 'DEMOGRAPHIC3' in raw_df.columns:
    raw_df['age'] = 2020 - raw_df['DEMOGRAPHIC3']
    raw_df.drop('DEMOGRAPHIC3', axis = 1, inplace=True)
    print(raw_df['age'])  

0       24
1       23
2       27
3       51
4       37
        ..
1206    22
1207    24
1208    20
1209    48
1210    68
Name: age, Length: 1211, dtype: int64


In [18]:
#Step 4: Identify the Ordinal Data 
cleaned_df=raw_df
neg_pos_dic1 = {"Don't Know":0, "Very Negative": 1, "Somewhat Negative":2, "Neutral":3, "Somewhat Positive":4, 
               "Very Positive":5}
 
cleaned_df["HR2_1"].replace(neg_pos_dic1, inplace=True) 
cleaned_df["HR2_2"].replace(neg_pos_dic1, inplace=True) 
cleaned_df["HR2_3"].replace(neg_pos_dic1, inplace=True) 
cleaned_df["HR2_4"].replace(neg_pos_dic1, inplace=True)  
cleaned_df["HR2_5"].replace(neg_pos_dic1, inplace=True)  
#cleaned_df.to_csv("step2_cleaned_df.csv")

dissat_satis_dic = {"No opinion": 0, "Very dissatisfied": 1, "Somewhat dissatisfied": 2, 
                    "Somewhat satisfied": 3, "Very satisfied": 4}
cleaned_df["HR3"].replace(dissat_satis_dic, inplace=True)
cleaned_df["HR5"].replace(dissat_satis_dic, inplace=True)

poor_excellent_dic = {"No opinion": 0,"Poor": 1, "Fair":2, "Good": 3, "Excellent": 4}
cleaned_df["HR4"].replace(poor_excellent_dic, inplace=True)
cleaned_df["HR6"].replace(poor_excellent_dic, inplace=True)

opp_favor_dic1 = {"Unsure": 0, "Strongly Oppose": 1, "Oppose": 2, "Favor": 3, "Strongly Favor": 4}
cleaned_df["HR7_1"].replace(opp_favor_dic1, inplace=True)
cleaned_df["HR7_2"].replace(opp_favor_dic1, inplace=True)
cleaned_df["HR7_3"].replace(opp_favor_dic1, inplace=True)
cleaned_df["HR7_4"].replace(opp_favor_dic1, inplace=True)

opp_favor_dic2 = {"Don't know": 0, "Strongly oppose": 1, "Somewhat oppose": 1, "Somewhat favor": 2, "Strongly favor": 3}
cleaned_df["HR7_1"].replace(opp_favor_dic2, inplace=True)
cleaned_df["HR7_2"].replace(opp_favor_dic2, inplace=True)
cleaned_df["HR7_3"].replace(opp_favor_dic2, inplace=True)
cleaned_df["HR7_4"].replace(opp_favor_dic2, inplace=True)
cleaned_df["HR8"].replace(opp_favor_dic2, inplace=True)

neg_pos_dic2 = {"No Opinion":0, "Mostly Negative": 1, "Mostly Positive":2}
cleaned_df["HR12_1"].replace(neg_pos_dic2, inplace=True) 
cleaned_df["HR12_2"].replace(neg_pos_dic2, inplace=True) 
cleaned_df["HR12_3"].replace(neg_pos_dic2, inplace=True) 
cleaned_df["HR12_4"].replace(neg_pos_dic2, inplace=True)  
cleaned_df["HR12_5"].replace(neg_pos_dic2, inplace=True)

adequate_dic1 = {"Don't know/unsure": 0, "Not adequate at all": 1, "Adequate": 2, "More than adequate": 3}
cleaned_df["STIMULUS3"].replace(adequate_dic1, inplace=True) 
cleaned_df["STIMULUS4"].replace(adequate_dic1, inplace=True)

likely_dic1 = {"Don't know/unsure": 0, "Highly unlikely": 1, "Unlikely": 2, "Likely": 3, "Highly likely": 4}
cleaned_df["PERSONAL3"].replace(likely_dic1, inplace=True)

disagree_agree_dic1 = {"Don't know/unsure": 0, "Strongly disagree": 1, "Disagree": 2, "Agree": 3, "Strongly agree": 4}
cleaned_df["PERSONAL4"].replace(disagree_agree_dic1, inplace=True)

worry_dic1 = {"Not worried at all": 0, "Not very worried": 1, "Somewhat Worried": 2, "Very Worried": 3}
cleaned_df["WORRY_1"].replace(worry_dic1, inplace=True)
cleaned_df["WORRY_2"].replace(worry_dic1, inplace=True)
cleaned_df["WORRY_3"].replace(worry_dic1, inplace=True)
cleaned_df["WORRY_4"].replace(worry_dic1, inplace=True)
cleaned_df["WORRY_5"].replace(worry_dic1, inplace=True)
cleaned_df["WORRY_6"].replace(worry_dic1, inplace=True)
cleaned_df["WORRY_7"].replace(worry_dic1, inplace=True)
cleaned_df["WORRY_8"].replace(worry_dic1, inplace=True)

often_dic1 = {"Very Little (less than weekly)": 0, "Somewhat (weekly)": 1, "A lot (daily)": 2, "Frequent (multiple times a day)": 3}
cleaned_df["MEDIA1"].replace(often_dic1, inplace=True)

confidence_dic1 = {"Unsure": 0, "No confidence at all": 1, "Very little confidence": 2, "Some confidence": 3,  "A great deal of confidence": 4,"Complete confidence": 5}
cleaned_df["MEDIA6_1"].replace(confidence_dic1, inplace=True)
cleaned_df["MEDIA6_2"].replace(confidence_dic1, inplace=True)
cleaned_df["MEDIA6_3"].replace(confidence_dic1, inplace=True)
cleaned_df["MEDIA6_4"].replace(confidence_dic1, inplace=True)
cleaned_df["MEDIA6_5"].replace(confidence_dic1, inplace=True)
cleaned_df["MEDIA6_6"].replace(confidence_dic1, inplace=True)
cleaned_df["MEDIA6_7"].replace(confidence_dic1, inplace=True)
 
bad_good_dic1 = {"Very bad": 0, "Bad": 1, "Fair": 2, "Good": 3, "Very good": 4}
cleaned_df["MEDIA7"].replace(bad_good_dic1, inplace=True)

disagree_agree_dic2 = {"Don't know": 0, "Strongly disagree": 1, "Somewhat disagree": 2, "Somewhat Agree": 3, "Strongly agree": 4}
cleaned_df["MEDIA10"].replace(disagree_agree_dic2, inplace=True)

smoker_dic1 = {"Never smoked": 0, "Former smoker, occasional": 1, "Former smoker, frequent": 2, 
               "Current Smoker, occasional": 3, "Current Smoker, frequent": 4}
cleaned_df["MEDIA11"].replace(smoker_dic1, inplace=True)
 
smoker_dic2 = {"Not at all": 0, "Some days": 1, "Every day": 2}
cleaned_df["MEDIA12"].replace(smoker_dic2, inplace=True) 

culture1_dic1 = {"Agree Strongly":5, "Agree":4, "Neither agree nor disagree":3, "Disagree":2, "Disagree Strongly":1}
cleaned_df["CULTURE1_1"].replace(culture1_dic1, inplace=True)
cleaned_df["CULTURE1_2"].replace(culture1_dic1, inplace=True) 
cleaned_df["CULTURE1_3"].replace(culture1_dic1, inplace=True) 
cleaned_df["CULTURE1_4"].replace(culture1_dic1, inplace=True) 
cleaned_df["CULTURE1_5"].replace(culture1_dic1, inplace=True) 
cleaned_df["CULTURE1_6"].replace(culture1_dic1, inplace=True)

culture2_dic1 = {"Agree Strongly":5, "Agree":4, "Neither agree nor disagree":3, "Disagree":2, "Disagree Strongly":1}
cleaned_df["CULTURE2_1"].replace(culture2_dic1, inplace=True)
cleaned_df["CULTURE2_2"].replace(culture2_dic1, inplace=True) 
cleaned_df["CULTURE2_3"].replace(culture2_dic1, inplace=True) 
cleaned_df["CULTURE2_4"].replace(culture2_dic1, inplace=True) 
cleaned_df["CULTURE2_5"].replace(culture2_dic1, inplace=True) 
cleaned_df["CULTURE2_6"].replace(culture2_dic1, inplace=True)
cleaned_df["CULTURE2_7"].replace(culture2_dic1, inplace=True)

cleaned_df.head() 

,ARM,HR1_10SCALE,HR2_1,HR2_2,HR2_3,HR2_4,HR2_5,HR3,HR4,HR5,...,DEMOGRAPHIC5,DEMOGRAPHIC6,DEMOGRAPHIC7,DEMOGRAPHIC8,DEMOGRAPHIC9,DEMOGRAPHIC10,DEMOGRAPHIC11,DEMOGRAPHIC12,DEMOGRAPHIC13,age
0,1,4,3,1,2,3,5,1,2,3,...,46825,Yes,No,Roman Catholic (Catholic),Somewhat important,Male,"$20,001-$50,000",White,Lower middle class,24
1,0,4,3,3,4,2,4,3,2,3,...,81650,Yes,No,"Protestant (Baptist, Methodist, Non-denominati...",Somewhat important,Female,"$10,001-$20,000",White,Lower middle class,23
2,2,10,0,0,0,0,0,0,0,0,...,31027,Yes,No,Something else (SPECIFY),Don't know/Refused,Female,"< $10,000",White,Lower class/poor,27
3,2,5,3,3,3,3,3,0,1,2,...,60609,Yes,No,Roman Catholic (Catholic),Not too important,Female,"$20,001-$50,000",White,Lower class/poor,51
4,1,5,3,0,1,1,2,0,4,1,...,39466,Yes,No,"Protestant (Baptist, Methodist, Non-denominati...",Not too important,Female,"< $10,000",White,Lower class/poor,37


In [19]:
#Step 5: For long long long text data, replace with survey answer index or A, B, C, D, E
#because the text is so long, dictionary contains key works to find in the text
#Clean up HR11
cleaned_df.loc[cleaned_df['HR11'].astype(str).str.startswith('Incrementally building on'),  'HR11'] = "A"
cleaned_df.loc[cleaned_df['HR11'].astype(str).str.startswith('Creating a universal'),  'HR11'] = "B"
cleaned_df.loc[cleaned_df['HR11'].astype(str).str.startswith('Reversing the Affordable'),  'HR11'] = "C"
cleaned_df.loc[cleaned_df['HR11'].astype(str).str.startswith('Something else'),  'HR11'] = "D"

#Clean up HR13
cleaned_df.loc[cleaned_df['HR13'].astype(str).str.startswith('Yes, I lost my health insurance due to'),  'HR13'] = "A"
cleaned_df.loc[cleaned_df['HR13'].astype(str).str.startswith('Yes, I lost my health insurance for'),  'HR13'] = "B"
cleaned_df.loc[cleaned_df['HR13'].astype(str).str.startswith('Yes, somebody close to me lost their health insurance due'),  
               'HR13'] = "C"
cleaned_df.loc[cleaned_df['HR13'].astype(str).str.startswith('Yes, somebody close to me lost their health insurance for'), 
               'HR13'] = "D"
cleaned_df.loc[cleaned_df['HR13'].astype(str).str.startswith('No'),  'HR13'] = "E"
cleaned_df.loc[cleaned_df['HR13'].astype(str).str.startswith('Unsure'),  'HR13'] = "F"

#Clean up HR14
cleaned_df.loc[cleaned_df['HR14'].astype(str).str.find('through an employer') >= 0,  'HR14'] = "A"
cleaned_df.loc[cleaned_df['HR14'].astype(str).str.find('with a subsidy') >= 0,  'HR14'] = "B"
cleaned_df.loc[cleaned_df['HR14'].astype(str).str.find('with no subsidy') >= 0, 'HR14'] = "C"
cleaned_df.loc[cleaned_df['HR14'].astype(str).str.find('None, uninsured') >= 0, 'HR14'] = "D"
cleaned_df.loc[cleaned_df['HR14'].astype(str).str.find('Medicaid') >= 0, 'HR14'] = "E"
cleaned_df.loc[cleaned_df['HR14'].astype(str).str.find('Medicare') >= 0, 'HR14'] = "F"
cleaned_df.loc[cleaned_df['HR14'].astype(str).str.find('Veteran') >= 0, 'HR14'] = "G"
cleaned_df.loc[cleaned_df['HR14'].astype(str).str.find('Other,') >= 0, 'HR14'] = "H"

#clean up COVID_HR1 a little bit
cleaned_df.loc[cleaned_df['COVID_HR1'].astype(str).str.startswith('Some-'),  'COVID_HR1'] = "Some"

#clean up COVID_HR2
cleaned_df.loc[cleaned_df['COVID_HR2'].astype(str).str.find('The government') >= 0,  'COVID_HR2'] = "A"
cleaned_df.loc[cleaned_df['COVID_HR2'].astype(str).str.find('Health insurance companies') >= 0,  'COVID_HR2'] = "B"
cleaned_df.loc[cleaned_df['COVID_HR2'].astype(str).str.find('A combination of both') >= 0, 'COVID_HR2'] = "C"
cleaned_df.loc[cleaned_df['COVID_HR2'].astype(str).str.find('Neither,') >= 0, 'COVID_HR2'] = "D"
cleaned_df.loc[cleaned_df['COVID_HR2'].astype(str).str.find('Something else') >= 0, 'COVID_HR2'] = "E"

#clean up COVID_HR3
cleaned_df.loc[cleaned_df['COVID_HR3'].astype(str).str.find('more favorable') >= 0,  'COVID_HR3'] = "A"
cleaned_df.loc[cleaned_df['COVID_HR3'].astype(str).str.find('less favorable') >= 0,  'COVID_HR3'] = "B"
cleaned_df.loc[cleaned_df['COVID_HR3'].astype(str).str.find('not affected') >= 0, 'COVID_HR3'] = "C" 

#clean up STIMULUS2
cleaned_df.loc[cleaned_df['STIMULUS2'].astype(str).str.find('somebody close to me') >= 0,  'STIMULUS2'] = "A"
cleaned_df.loc[cleaned_df['STIMULUS2'].astype(str).str.find('immediate family') >= 0,  'STIMULUS2'] = "B"
cleaned_df.loc[cleaned_df['STIMULUS2'].astype(str).str.find('No') >= 0, 'STIMULUS2'] = "C"
 
#clean up STIMULUS5
cleaned_df.loc[cleaned_df['STIMULUS5'].astype(str).str.find('check from the government') >= 0,  'STIMULUS5'] = "A"
cleaned_df.loc[cleaned_df['STIMULUS5'].astype(str).str.find('small business loan') >= 0,  'STIMULUS5'] = "B"
cleaned_df.loc[cleaned_df['STIMULUS5'].astype(str).str.find('employment benefits') >= 0, 'STIMULUS5'] = "C"
cleaned_df.loc[cleaned_df['STIMULUS5'].astype(str).str.find('No') >= 0,  'STIMULUS5'] = "D"
cleaned_df.loc[cleaned_df['STIMULUS5'].astype(str).str.find('Unsure') >= 0, 'STIMULUS5'] = "E"
cleaned_df.loc[cleaned_df['STIMULUS5'].astype(str).str.find('Other') >= 0, 'STIMULUS5'] = "F"

#clean up STIMULUS6
cleaned_df.loc[cleaned_df['STIMULUS6'].astype(str).str.find('market run its course') >= 0,  'STIMULUS6'] = "A"
cleaned_df.loc[cleaned_df['STIMULUS6'].astype(str).str.find('direct payments') >= 0,  'STIMULUS6'] = "B"
cleaned_df.loc[cleaned_df['STIMULUS6'].astype(str).str.find('stimulus for small') >= 0, 'STIMULUS6'] = "C"
cleaned_df.loc[cleaned_df['STIMULUS6'].astype(str).str.find('Guarantee the salary') >= 0,  'STIMULUS6'] = "D"
cleaned_df.loc[cleaned_df['STIMULUS6'].astype(str).str.find('Other,') >= 0, 'STIMULUS6'] = "E"

#clean up policy1
cleaned_df.loc[cleaned_df['POLICY1'].astype(str).str.find('Not at all') >= 0,  'POLICY1'] = 0
cleaned_df.loc[cleaned_df['POLICY1'].astype(str).str.find('A little') >= 0,  'POLICY1'] = 1
cleaned_df.loc[cleaned_df['POLICY1'].astype(str).str.find('A lot') >= 0, 'POLICY1'] = 2 
 
#clean up policy2
cleaned_df.loc[cleaned_df['POLICY2'].astype(str).str.find('Not at all') >= 0,  'POLICY2'] = 0
cleaned_df.loc[cleaned_df['POLICY2'].astype(str).str.find('Somewhat concerned') >= 0,  'POLICY2'] = 1
cleaned_df.loc[cleaned_df['POLICY2'].astype(str).str.find('Very concerned') >= 0, 'POLICY2'] = 2

#clean up policy3
cleaned_df.loc[cleaned_df['POLICY3'].astype(str).str.find('overreaction') >= 0,  'POLICY3'] = "A"
cleaned_df.loc[cleaned_df['POLICY3'].astype(str).str.find('serious illness') >= 0,  'POLICY3'] = "B"
cleaned_df.loc[cleaned_df['POLICY3'].astype(str).str.find('large threat') >= 0, 'POLICY3'] = "C" 

#clean up policy4
cleaned_df.loc[cleaned_df['POLICY4'].astype(str).str.find('economy is a bigger threat') >= 0,  'POLICY4'] = "A"
cleaned_df.loc[cleaned_df['POLICY4'].astype(str).str.find('People should not put economic') >= 0,  'POLICY4'] = "B"
 
#clean up Personal2
cleaned_df.loc[cleaned_df['PERSONAL2'].astype(str).str.find('None of the time') >= 0,  'PERSONAL2'] = 0
cleaned_df.loc[cleaned_df['PERSONAL2'].astype(str).str.find('Some of the time') >= 0,  'PERSONAL2'] = 1
cleaned_df.loc[cleaned_df['PERSONAL2'].astype(str).str.find('Most of the time') >= 0, 'PERSONAL2'] = 2 
cleaned_df.loc[cleaned_df['PERSONAL2'].astype(str).str.find('All of the time') >= 0, 'PERSONAL2'] = 3 

#clean up MEDIA2
cleaned_df.loc[cleaned_df['MEDIA2'].astype(str).str.find('began in China') >= 0,  'MEDIA2'] = "A"
cleaned_df.loc[cleaned_df['MEDIA2'].astype(str).str.find('began in Italy') >= 0,  'MEDIA2'] = "B"
cleaned_df.loc[cleaned_df['MEDIA2'].astype(str).str.find('began in the US') >= 0, 'MEDIA2'] = "C"
cleaned_df.loc[cleaned_df['MEDIA2'].astype(str).str.find('not paying close attention') >= 0,  'MEDIA2'] = "D"

#clean up MEDIA5
cleaned_df.loc[cleaned_df['MEDIA5'].astype(str).str.find('Fox News') >= 0,  'MEDIA5'] = "A"
cleaned_df.loc[cleaned_df['MEDIA5'].astype(str).str.find('MSNBC') >= 0,  'MEDIA5'] = "B"
cleaned_df.loc[cleaned_df['MEDIA5'].astype(str).str.find('CNN') >= 0, 'MEDIA5'] = "C"
cleaned_df.loc[cleaned_df['MEDIA5'].astype(str).str.find('Evening news or morning shows') >= 0,  'MEDIA5'] = "D"
cleaned_df.loc[cleaned_df['MEDIA5'].astype(str).str.find('Alternative news media outlets') >= 0, 'MEDIA5'] = "E"
cleaned_df.loc[cleaned_df['MEDIA5'].astype(str).str.find('Other') >= 0,  'MEDIA5'] = "F"

#clean up Personal2
cleaned_df.loc[cleaned_df['MEDIA10'].astype(str).str.find("Don't know") >= 0,  'MEDIA10'] = 0
cleaned_df.loc[cleaned_df['MEDIA10'].astype(str).str.find("Strongly Disagree") >= 0,  'MEDIA10'] = 1
cleaned_df.loc[cleaned_df['MEDIA10'].astype(str).str.find("Somewhat Disagree") >= 0, 'MEDIA10'] = 2 
cleaned_df.loc[cleaned_df['MEDIA10'].astype(str).str.find("Somewhat Agree") >= 0, 'MEDIA10'] = 3 
cleaned_df.loc[cleaned_df['MEDIA10'].astype(str).str.find("Strongly Agree") >= 0,  'MEDIA10'] = 4
 
#clean up Media13
cleaned_df.loc[cleaned_df['MEDIA13'].astype(str).str.find('Yes') >= 0,  'MEDIA13'] = "A"
cleaned_df.loc[cleaned_df['MEDIA13'].astype(str).str.find('No') >= 0,  'MEDIA13'] = "B"
cleaned_df.loc[cleaned_df['MEDIA13'].astype(str).str.find("Don't know") >= 0, 'MEDIA13'] = "C"

#clean up Media14
cleaned_df.loc[cleaned_df['MEDIA14'].astype(str).str.find('Yes') >= 0,  'MEDIA15'] = "A"
cleaned_df.loc[cleaned_df['MEDIA14'].astype(str).str.find('No') >= 0,  'MEDIA15'] = "B"

#clean up MEDIA15
cleaned_df.loc[cleaned_df['MEDIA15'].astype(str).str.find('test was positive') >= 0,  'MEDIA15'] = "A"
cleaned_df.loc[cleaned_df['MEDIA15'].astype(str).str.find('test was negative') >= 0,  'MEDIA15'] = "B"
cleaned_df.loc[cleaned_df['MEDIA15'].astype(str).str.find('could not get a test') >= 0, 'MEDIA15'] = "C"
cleaned_df.loc[cleaned_df['MEDIA15'].astype(str).str.find('not tried to be tested') >= 0,  'MEDIA15'] = "D" 

#clean up Media17
cleaned_df.loc[cleaned_df['MEDIA17'].astype(str).str.find('Yes') >= 0,  'MEDIA17'] = "A"
cleaned_df.loc[cleaned_df['MEDIA17'].astype(str).str.find('No') >= 0,  'MEDIA17'] = "B"

#clean up Media9 
cleaned_df.loc[cleaned_df['MEDIA9'].astype(str).str.find('Yes') >= 0,  'MEDIA9'] = "A"
cleaned_df.loc[cleaned_df['MEDIA9'].astype(str).str.find('No') >= 0,  'MEDIA9'] = "B"
cleaned_df.loc[cleaned_df['MEDIA9'].astype(str).str.find("Don't remember") >= 0, 'MEDIA9'] = "C"
    
#clean up MH1
cleaned_df.loc[cleaned_df['MH1'].astype(str).str.find('Not at all') >= 0,  'MH1'] = 0
cleaned_df.loc[cleaned_df['MH1'].astype(str).str.find('Several days') >= 0,  'MH1'] = 1
cleaned_df.loc[cleaned_df['MH1'].astype(str).str.find('More than half the days') >= 0, 'MH1'] = 2
cleaned_df.loc[cleaned_df['MH1'].astype(str).str.find('Nearly every day') >= 0, 'MH1'] = 3

#clean up MH2
cleaned_df.loc[cleaned_df['MH2'].astype(str).str.find('Not at all') >= 0,  'MH2'] = 0
cleaned_df.loc[cleaned_df['MH2'].astype(str).str.find('Several days') >= 0,  'MH2'] = 1
cleaned_df.loc[cleaned_df['MH2'].astype(str).str.find('More than half the days') >= 0, 'MH2'] = 2
cleaned_df.loc[cleaned_df['MH2'].astype(str).str.find('Nearly every day') >= 0, 'MH2'] = 3

#clean up CHILDCARE2
cleaned_df.loc[cleaned_df['CHILDCARE2'].astype(str).str.find('Yes') >= 0,  'CHILDCARE2'] = "A"
cleaned_df.loc[cleaned_df['CHILDCARE2'].astype(str).str.find('No') >= 0,  'CHILDCARE2'] = "B"
cleaned_df.loc[cleaned_df['CHILDCARE2'].astype(str).str.find("Don't Know") >= 0, 'CHILDCARE2'] = "C" 

#clean up IDEOLOGY1
cleaned_df.loc[cleaned_df['IDEOLOGY1'].astype(str).str.find('Democrat') >= 0,  'IDEOLOGY1'] = "A"
cleaned_df.loc[cleaned_df['IDEOLOGY1'].astype(str).str.find('Republican') >= 0,  'IDEOLOGY1'] = "B"
cleaned_df.loc[cleaned_df['IDEOLOGY1'].astype(str).str.find('Independent') >= 0, 'IDEOLOGY1'] = "C" 
cleaned_df.loc[cleaned_df['IDEOLOGY1'].astype(str).str.find('Other,') >= 0, 'IDEOLOGY1'] = "D"

#clean up IDEOLOGY2
cleaned_df.loc[cleaned_df['IDEOLOGY2'].astype(str).str.find('Democrat') >= 0,  'IDEOLOGY2'] = "A"
cleaned_df.loc[cleaned_df['IDEOLOGY2'].astype(str).str.find('Republican') >= 0,  'IDEOLOGY2'] = "B"
cleaned_df.loc[cleaned_df['IDEOLOGY2'].astype(str).str.find('Tea Party') >= 0, 'IDEOLOGY2'] = "C" 
cleaned_df.loc[cleaned_df['IDEOLOGY2'].astype(str).str.find('Libertarian Party') >= 0, 'IDEOLOGY2'] = "D"
cleaned_df.loc[cleaned_df['IDEOLOGY2'].astype(str).str.find('Working Families') >= 0, 'IDEOLOGY2'] = "E" 
cleaned_df.loc[cleaned_df['IDEOLOGY2'].astype(str).str.find('Green Party') >= 0, 'IDEOLOGY2'] = "F"
cleaned_df.loc[cleaned_df['IDEOLOGY2'].astype(str).str.find('Socialist Party of America') >= 0, 'IDEOLOGY2'] = "G" 
cleaned_df.loc[cleaned_df['IDEOLOGY2'].astype(str).str.find('None') >= 0, 'IDEOLOGY2'] = "H"
cleaned_df.loc[cleaned_df['IDEOLOGY2'].astype(str).str.find('Other,') >= 0, 'IDEOLOGY2'] = "I"

#clean up IDEOLOGY3
cleaned_df.loc[cleaned_df['IDEOLOGY3'].astype(str).str.find('Yes') >= 0,  'IDEOLOGY3'] = "A"
cleaned_df.loc[cleaned_df['IDEOLOGY3'].astype(str).str.find('No, not planning') >= 0,  'IDEOLOGY3'] = "B"
cleaned_df.loc[cleaned_df['IDEOLOGY3'].astype(str).str.find('Primary was') >= 0, 'IDEOLOGY3'] = "C" 
cleaned_df.loc[cleaned_df['IDEOLOGY3'].astype(str).str.find("Don't know") >= 0, 'IDEOLOGY3'] = "D"

#clean up IDEOLOGY4
cleaned_df.loc[cleaned_df['IDEOLOGY4'].astype(str).str.find('Bernie Sanders') >= 0,  'IDEOLOGY4'] = "A"
cleaned_df.loc[cleaned_df['IDEOLOGY4'].astype(str).str.find('Joe Biden') >= 0,  'IDEOLOGY4'] = "B"
cleaned_df.loc[cleaned_df['IDEOLOGY4'].astype(str).str.find('Someone else') >= 0, 'IDEOLOGY4'] = "C" 

#clean up IDEOLOGY5
cleaned_df.loc[cleaned_df['IDEOLOGY5'].astype(str).str.find('Far Right') >= 0,  'IDEOLOGY5'] = 2
cleaned_df.loc[cleaned_df['IDEOLOGY5'].astype(str).str.find('Center Right') >= 0,  'IDEOLOGY5'] = 1
cleaned_df.loc[cleaned_df['IDEOLOGY5'].astype(str).str.find('Neither Right') >= 0, 'IDEOLOGY5'] = 0
cleaned_df.loc[cleaned_df['IDEOLOGY5'].astype(str).str.find('Center Left') >= 0,  'IDEOLOGY5'] = -1
cleaned_df.loc[cleaned_df['IDEOLOGY5'].astype(str).str.find('Far Left') >= 0, 'IDEOLOGY5'] = -2

#clean up IDEOLOGY6
cleaned_df.loc[cleaned_df['IDEOLOGY6'].astype(str).str.find('Deeply') >= 0,  'IDEOLOGY6'] = 4
cleaned_df.loc[cleaned_df['IDEOLOGY6'].astype(str).str.find('Somewhat political') >= 0,  'IDEOLOGY6'] = 3
cleaned_df.loc[cleaned_df['IDEOLOGY6'].astype(str).str.find('Neither') >= 0, 'IDEOLOGY6'] = 2
cleaned_df.loc[cleaned_df['IDEOLOGY6'].astype(str).str.find('Somewhat unpolitical') >= 0,  'IDEOLOGY6'] = 1
cleaned_df.loc[cleaned_df['IDEOLOGY6'].astype(str).str.find('Not political') >= 0, 'IDEOLOGY6'] = 0

#clean up IDEOLOGY7
cleaned_df.loc[cleaned_df['IDEOLOGY7'].astype(str).str.find('Did not vote') >= 0,  'IDEOLOGY7'] = "A"
cleaned_df.loc[cleaned_df['IDEOLOGY7'].astype(str).str.find('Was not eligible to') >= 0,  'IDEOLOGY7'] = "B"
cleaned_df.loc[cleaned_df['IDEOLOGY7'].astype(str).str.find('Donald Trump') >= 0, 'IDEOLOGY7'] = "C"
cleaned_df.loc[cleaned_df['IDEOLOGY7'].astype(str).str.find('Hillary Clinton') >= 0,  'IDEOLOGY7'] = "D"
cleaned_df.loc[cleaned_df['IDEOLOGY7'].astype(str).str.find('Someone else') >= 0, 'IDEOLOGY7'] = "E"
cleaned_df.loc[cleaned_df['IDEOLOGY7'].astype(str).str.find("Don't remember") >= 0, 'IDEOLOGY7'] = "F"

#clean up IDEOLOGY8
cleaned_df.loc[cleaned_df['IDEOLOGY8'].astype(str).str.find('Not planning to vote') >= 0,  'IDEOLOGY8'] = "A"
cleaned_df.loc[cleaned_df['IDEOLOGY8'].astype(str).str.find('Not eligible to vote') >= 0,  'IDEOLOGY8'] = "B"
cleaned_df.loc[cleaned_df['IDEOLOGY8'].astype(str).str.find('Donald Trump') >= 0, 'IDEOLOGY8'] = "C"
cleaned_df.loc[cleaned_df['IDEOLOGY8'].astype(str).str.find('Joe Biden') >= 0,  'IDEOLOGY8'] = "D"
cleaned_df.loc[cleaned_df['IDEOLOGY8'].astype(str).str.find('Have not yet') >= 0, 'IDEOLOGY8'] = "E"
cleaned_df.loc[cleaned_df['IDEOLOGY8'].astype(str).str.find("Write in other") >= 0, 'IDEOLOGY8'] = "F"

#clean up RUMORS2
cleaned_df.loc[cleaned_df['RUMORS2'].astype(str).str.find('accidently') >= 0,  'RUMORS2'] = "A"
cleaned_df.loc[cleaned_df['RUMORS2'].astype(str).str.find('purposefully') >= 0,  'RUMORS2'] = "B"
cleaned_df.loc[cleaned_df['RUMORS2'].astype(str).str.find('The US army') >= 0, 'RUMORS2'] = "C"
cleaned_df.loc[cleaned_df['RUMORS2'].astype(str).str.find('bat to humans') >= 0,  'RUMORS2'] = "D"
cleaned_df.loc[cleaned_df['RUMORS2'].astype(str).str.find('bats to another animal') >= 0, 'RUMORS2'] = "E"
cleaned_df.loc[cleaned_df['RUMORS2'].astype(str).str.find('It was introduced in a') >= 0, 'RUMORS2'] = "F"
cleaned_df.loc[cleaned_df['RUMORS2'].astype(str).str.find('Other,') >= 0, 'RUMORS2'] = "G"

#clean up DEMOGRAPHIC8
cleaned_df.loc[cleaned_df['DEMOGRAPHIC8'].astype(str).str.find('Protestant') >= 0,  'DEMOGRAPHIC8'] = "A"
cleaned_df.loc[cleaned_df['DEMOGRAPHIC8'].astype(str).str.find('Roman') >= 0,  'DEMOGRAPHIC8'] = "B"
cleaned_df.loc[cleaned_df['DEMOGRAPHIC8'].astype(str).str.find('Mormon') >= 0, 'DEMOGRAPHIC8'] = "C"
cleaned_df.loc[cleaned_df['DEMOGRAPHIC8'].astype(str).str.find('Orthodox') >= 0,  'DEMOGRAPHIC8'] = "D"
cleaned_df.loc[cleaned_df['DEMOGRAPHIC8'].astype(str).str.find('Jewish') >= 0, 'DEMOGRAPHIC8'] = "E"
cleaned_df.loc[cleaned_df['DEMOGRAPHIC8'].astype(str).str.find('Muslim') >= 0, 'DEMOGRAPHIC8'] = "F"
cleaned_df.loc[cleaned_df['DEMOGRAPHIC8'].astype(str).str.find('Buddhist') >= 0, 'DEMOGRAPHIC8'] = "G"
cleaned_df.loc[cleaned_df['DEMOGRAPHIC8'].astype(str).str.find('Hindu') >= 0, 'DEMOGRAPHIC8'] = "H"
cleaned_df.loc[cleaned_df['DEMOGRAPHIC8'].astype(str).str.find('Atheist') >= 0, 'DEMOGRAPHIC8'] = "I"
cleaned_df.loc[cleaned_df['DEMOGRAPHIC8'].astype(str).str.find('Agnostic') >= 0, 'DEMOGRAPHIC8'] = "J"
cleaned_df.loc[cleaned_df['DEMOGRAPHIC8'].astype(str).str.find('Agnostic') >= 0, 'DEMOGRAPHIC8'] = "K"
cleaned_df.loc[cleaned_df['DEMOGRAPHIC8'].astype(str).str.find('Something') >= 0, 'DEMOGRAPHIC8'] = "L"

#clean up DEMOGRAPHIC9
cleaned_df.loc[cleaned_df['DEMOGRAPHIC9'].astype(str).str.find('Very important') >= 0,  'DEMOGRAPHIC9'] = 4
cleaned_df.loc[cleaned_df['DEMOGRAPHIC9'].astype(str).str.find('Somewhat important') >= 0,  'DEMOGRAPHIC9'] = 3
cleaned_df.loc[cleaned_df['DEMOGRAPHIC9'].astype(str).str.find('Not too important') >= 0, 'DEMOGRAPHIC9'] = 2
cleaned_df.loc[cleaned_df['DEMOGRAPHIC9'].astype(str).str.find('Not at all important') >= 0,  'DEMOGRAPHIC9'] = 1
cleaned_df.loc[cleaned_df['DEMOGRAPHIC9'].astype(str).str.find("Don't know/Refused") >= 0, 'DEMOGRAPHIC9'] = 0

#clean up DEMOGRAPHIC11
cleaned_df.loc[cleaned_df['DEMOGRAPHIC11'].astype(str).str.find('< $10,000') >= 0,  'DEMOGRAPHIC11'] = 0
cleaned_df.loc[cleaned_df['DEMOGRAPHIC11'].astype(str).str.find('$10,001-') >= 0,  'DEMOGRAPHIC11'] = 1
cleaned_df.loc[cleaned_df['DEMOGRAPHIC11'].astype(str).str.find('$20,001-') >= 0, 'DEMOGRAPHIC11'] = 2
cleaned_df.loc[cleaned_df['DEMOGRAPHIC11'].astype(str).str.find('$50,001-') >= 0,  'DEMOGRAPHIC11'] = 3
cleaned_df.loc[cleaned_df['DEMOGRAPHIC11'].astype(str).str.find('$75,001-') >= 0, 'DEMOGRAPHIC11'] = 4
cleaned_df.loc[cleaned_df['DEMOGRAPHIC11'].astype(str).str.find('$150,001-') >= 0, 'DEMOGRAPHIC11'] = 5
cleaned_df.loc[cleaned_df['DEMOGRAPHIC11'].astype(str).str.find('>$201,000-') >= 0, 'DEMOGRAPHIC11'] = 6
cleaned_df.loc[cleaned_df['DEMOGRAPHIC11'].astype(str).str.find('$250,001+') >= 0, 'DEMOGRAPHIC11'] = 7

#clean up DEMOGRAPHIC12
cleaned_df.loc[cleaned_df['DEMOGRAPHIC12'].astype(str).str.find('White') >= 0,  'DEMOGRAPHIC12'] = "A"
cleaned_df.loc[cleaned_df['DEMOGRAPHIC12'].astype(str).str.find('Hispanic') >= 0,  'DEMOGRAPHIC12'] = "B"
cleaned_df.loc[cleaned_df['DEMOGRAPHIC12'].astype(str).str.find('Black') >= 0, 'DEMOGRAPHIC12'] = "C"
cleaned_df.loc[cleaned_df['DEMOGRAPHIC12'].astype(str).str.find('Asian') >= 0,  'DEMOGRAPHIC12'] = "D"
cleaned_df.loc[cleaned_df['DEMOGRAPHIC12'].astype(str).str.find('American') >= 0, 'DEMOGRAPHIC12'] = "E"
cleaned_df.loc[cleaned_df['DEMOGRAPHIC12'].astype(str).str.find('Middle Eastern') >= 0, 'DEMOGRAPHIC12'] = "F"
cleaned_df.loc[cleaned_df['DEMOGRAPHIC12'].astype(str).str.find('Native Hawaiian') >= 0, 'DEMOGRAPHIC12'] = "G"
cleaned_df.loc[cleaned_df['DEMOGRAPHIC12'].astype(str).str.find('Some other Race') >= 0, 'DEMOGRAPHIC12'] = "H"

#clean up DEMOGRAPHIC13
cleaned_df.loc[cleaned_df['DEMOGRAPHIC13'].astype(str).str.find('Lower') >= 0,  'DEMOGRAPHIC13'] = 0
cleaned_df.loc[cleaned_df['DEMOGRAPHIC13'].astype(str).str.find('Middle') >= 0,  'DEMOGRAPHIC13'] = 1
cleaned_df.loc[cleaned_df['DEMOGRAPHIC13'].astype(str).str.find('Upper middle') >= 0, 'DEMOGRAPHIC13'] = 2
cleaned_df.loc[cleaned_df['DEMOGRAPHIC13'].astype(str).str.find('Upper class') >= 0,  'DEMOGRAPHIC13'] = 3


#def replaceLongText (string_data, long_text_dic):
#    for key, value in long_text_dic.items(): 
#        if string_data.astype(str).str.find(key) >= 0:
#            return value
            
#HR11_dic = {"Incrementally building on":"A", "Creating a universal": "B", 
#            "Reversing the Affordable": "C", "Something else,":"D"}

#Not working 
#cleaned_df['HR11'] = (cleaned_df.apply(lambda x: replaceLongText(x[x['HR11']], HR11_dic), axis=1))


In [20]:
#For multiple answer questions, need create extra columns for every question
#If you run the second time, since column '' has been dropped, you will see the error message, add checking column 
if 'STIMULUS1' in cleaned_df.columns:
    cleaned_df['STIMULUS1_WORK_FROM_HOME'] = cleaned_df['STIMULUS1'].apply(lambda x: 1 if x.find('Work from home') >= 0 else 0) 
    cleaned_df['STIMULUS1_LOST_JOB'] = cleaned_df['STIMULUS1'].apply(lambda x: 1 if x.find('Lost job') >= 0 else 0) 
    cleaned_df['STIMULUS1_FURLOUGHED'] = cleaned_df['STIMULUS1'].apply(lambda x: 1 if x.find('Furloughed') >= 0 else 0) 
    cleaned_df['STIMULUS1_REDUCED_HOURS'] = cleaned_df['STIMULUS1'].apply(lambda x: 1 if x.find('Reduced hours') >= 0 else 0) 
    cleaned_df['STIMULUS1_PAY_CUT'] = cleaned_df['STIMULUS1'].apply(lambda x: 1 if x.find('Pay cut') >= 0 else 0) 
    cleaned_df['STIMULUS1_WORK_MORE_HOUR'] = cleaned_df['STIMULUS1'].apply(lambda x: 1 if x.find('Work more hours') >= 0 else 0) 
    cleaned_df['STIMULUS1_INCREASED_PAY'] = cleaned_df['STIMULUS1'].apply(lambda x: 1 if x.find('Increased pay') >= 0 else 0) 
    cleaned_df['STIMULUS1_Nothing changed'] = cleaned_df['STIMULUS1'].apply(lambda x: 1 if x.find('Nothing changed') >= 0 else 0) 
    cleaned_df.drop('STIMULUS1', axis = 1, inplace=True) 

if 'POLICY5' in cleaned_df.columns:
    cleaned_df['POLICY5_EVERYONE_SD'] = cleaned_df['POLICY5'].apply(lambda x: 1 if x.find('Require everyone') >= 0 else 0) 
    cleaned_df['POLICY5_AFFECTED_AREA_SD'] = cleaned_df['POLICY5'].apply(lambda x: 1 if x.find('heavily affected areas') >= 0 else 0) 
    cleaned_df['POLICY5_VOLUNTARY_SD'] = cleaned_df['POLICY5'].apply(lambda x: 1 if x.find('voluntary social distancing') >= 0 else 0) 
    cleaned_df['POLICY5_OLD_PEOPLE_SD'] = cleaned_df['POLICY5'].apply(lambda x: 1 if x.find('older people age 66') >= 0 else 0) 
    cleaned_df['POLICY5_CLOSE_SCHOOL'] = cleaned_df['POLICY5'].apply(lambda x: 1 if x.find('Close schools') >= 0 else 0) 
    cleaned_df['POLICY5_CLOSE_UNIVERSITY'] = cleaned_df['POLICY5'].apply(lambda x: 1 if x.find('Close universities') >= 0 else 0) 
    cleaned_df['POLICY5_CANCEL_LARGE_EVENT'] = cleaned_df['POLICY5'].apply(lambda x: 1 if x.find('Cancel large events') >= 0 else 0) 
    cleaned_df['POLICY5_OTHER'] = cleaned_df['POLICY5'].apply(lambda x: 1 if x.find('OTHER') >= 0 else 0) 
    cleaned_df.drop('POLICY5', axis = 1, inplace=True) 
    
if 'POLICY6' in cleaned_df.columns:
    cleaned_df['POLICY6_EVERYONE_SD'] = cleaned_df['POLICY6'].apply(lambda x: 1 if x.find('practice social distancing') >= 0 else 0) 
    cleaned_df['POLICY6_AFFECTED_AREA_SD'] = cleaned_df['POLICY6'].apply(lambda x: 1 if x.find('heavily affected areas') >= 0 else 0) 
    cleaned_df['POLICY6_VOLUNTARY_SD'] = cleaned_df['POLICY6'].apply(lambda x: 1 if x.find('voluntary social distancing') >= 0 else 0) 
    cleaned_df['POLICY6_OLD_PEOPLE_SD'] = cleaned_df['POLICY6'].apply(lambda x: 1 if x.find('older people age 66') >= 0 else 0) 
    cleaned_df['POLICY6_CLOSE_SCHOOL'] = cleaned_df['POLICY6'].apply(lambda x: 1 if x.find('Close schools') >= 0 else 0) 
    cleaned_df['POLICY6_CLOSE_UNIVERSITY'] = cleaned_df['POLICY6'].apply(lambda x: 1 if x.find('Close universities') >= 0 else 0) 
    cleaned_df['POLICY6_CANCEL_LARGE_EVENT'] = cleaned_df['POLICY6'].apply(lambda x: 1 if x.find('Cancel large events') >= 0 else 0) 
    cleaned_df['POLICY6_OTHER'] = cleaned_df['POLICY6'].apply(lambda x: 1 if x.find('OTHER') >= 0 else 0) 
    cleaned_df.drop('POLICY6', axis = 1, inplace=True) 

if 'PERSONAL1' in cleaned_df.columns: 
    cleaned_df['PERSONAL1_WFH'] = cleaned_df['PERSONAL1'].apply(lambda x: 1 if x.find('Working from home') >= 0 else 0) 
    cleaned_df['PERSONAL1_STAY_HOME'] = cleaned_df['PERSONAL1'].apply(lambda x: 1 if x.find('Staying home and only going out') >= 0 else 0) 
    cleaned_df['PERSONAL1_DISTANCE_OLD_PEOPLE'] = cleaned_df['PERSONAL1'].apply(lambda x: 1 if x.find('older relatives') >= 0 else 0) 
    cleaned_df['PERSONAL1_DISTANCE_FAM_MEMBERS'] = cleaned_df['PERSONAL1'].apply(lambda x: 1 if x.find('other family members') >= 0 else 0) 
    cleaned_df['PERSONAL1_KEEP_KIDS_HOME1'] = cleaned_df['PERSONAL1'].apply(lambda x: 1 if x.find('because they are closed') >= 0 else 0) 
    cleaned_df['PERSONAL1_KEEP_KIDS_HOME2'] = cleaned_df['PERSONAL1'].apply(lambda x: 1 if x.find('daycare voluntarily') >= 0 else 0) 
    cleaned_df.drop('PERSONAL1', axis = 1, inplace=True)     
   
 
if 'PERSONAL5' in cleaned_df.columns: 
    cleaned_df['PERSONAL5_WASH_HAND_OFTEN'] = cleaned_df['PERSONAL5'].apply(lambda x: 1 if x.find('Wash hands more often') >= 0 else 0) 
    cleaned_df['PERSONAL5_USE_HAND_SANITIZER'] = cleaned_df['PERSONAL5'].apply(lambda x: 1 if x.find('Used hand sanitizer') >= 0 else 0) 
    cleaned_df['PERSONAL5_TEST_COVID'] = cleaned_df['PERSONAL5'].apply(lambda x: 1 if x.find('Sought out testing for Coronavirus') >= 0 else 0) 
    cleaned_df['PERSONAL5_CHANGE_TRAVEL'] = cleaned_df['PERSONAL5'].apply(lambda x: 1 if x.find('Changed travel plans') >= 0 else 0) 
    cleaned_df['PERSONAL5_AVOID_CONTACT_OTHERS'] = cleaned_df['PERSONAL5'].apply(lambda x: 1 if x.find('Avoided contact with others') >= 0 else 0) 
    cleaned_df['PERSONAL5_AVOID_OLDER_ADULTS'] = cleaned_df['PERSONAL5'].apply(lambda x: 1 if x.find('Avoiding contact with older adults') >= 0 else 0) 
    cleaned_df['PERSONAL5_AVOID_GATHERING'] = cleaned_df['PERSONAL5'].apply(lambda x: 1 if x.find('Avoided gatherings') >= 0 else 0) 
    cleaned_df['PERSONAL5_SOUGHT_INFO'] = cleaned_df['PERSONAL5'].apply(lambda x: 1 if x.find('Sought information on Coronavirus') >= 0 else 0) 
    cleaned_df['PERSONAL5_SELF_QUARANTINED'] = cleaned_df['PERSONAL5'].apply(lambda x: 1 if x.find('Self-quarantined/isolated') >= 0 else 0) 
    cleaned_df.drop('PERSONAL5', axis = 1, inplace=True)  
    
if 'PERSONAL6' in cleaned_df.columns: 
    cleaned_df['PERSONAL6_SELF_COMPLICATION'] = cleaned_df['PERSONAL6'].apply(lambda x: 1 if x.find('Experiencing serious complication') >= 0 else 0) 
    cleaned_df['PERSONAL6_SOMEONE_COMPLICATION'] = cleaned_df['PERSONAL6'].apply(lambda x: 1 if x.find('Someone I care about experiencing') >= 0 else 0) 
    cleaned_df['PERSONAL6_NO_FOOD'] = cleaned_df['PERSONAL6'].apply(lambda x: 1 if x.find('Not being able to put food') >= 0 else 0) 
    cleaned_df['PERSONAL6_FEELING_ANXIOUS'] = cleaned_df['PERSONAL6'].apply(lambda x: 1 if x.find('Feeling isolated, anxious') >= 0 else 0) 
    cleaned_df['PERSONAL6_LACK_CARE'] = cleaned_df['PERSONAL6'].apply(lambda x: 1 if x.find('Not being able to get needed care') >= 0 else 0)
    cleaned_df['PERSONAL6_LACK_ACCESS'] = cleaned_df['PERSONAL6'].apply(lambda x: 1 if x.find('Not being able to access care') >= 0 else 0) 
    cleaned_df['PERSONAL6_KIDS_EDU'] =  cleaned_df['PERSONAL6'].apply(lambda x: 1 if x.find('education') >= 0 else 0)   
    cleaned_df['PERSONAL6_CANNOT_PAY_RENT'] = cleaned_df['PERSONAL6'].apply(lambda x: 1 if x.find('pay the rent') >= 0 else 0) 
    cleaned_df['PERSONAL6_CANNOT_TAKE_CARE_OTHER'] = cleaned_df['PERSONAL6'].apply(lambda x: 1 if x.find('take care of family members') >= 0 else 0) 
    cleaned_df['PERSONAL6_CANNOT_WORK'] = cleaned_df['PERSONAL6'].apply(lambda x: 1 if x.find('Not being able to work') >= 0 else 0) 
    cleaned_df['PERSONAL6_LONG_TERM_FINANCIAL'] = cleaned_df['PERSONAL6'].apply(lambda x: 1 if x.find('Long term financial impacts') >= 0 else 0) 
    cleaned_df['PERSONAL6_OTHER'] = cleaned_df['PERSONAL6'].apply(lambda x: 1 if x.find('Other,') >= 0 else 0) 
    cleaned_df.drop('PERSONAL6', axis = 1, inplace=True)  
    
if 'PERSONAL7' in cleaned_df.columns:      
    cleaned_df['PERSONAL7_REDUCED_INCOME'] = cleaned_df['PERSONAL7'].apply(lambda x: 1 if x.find('Reduced income') >= 0 else 0) 
    cleaned_df['PERSONAL7_LOST_MY_JOB'] = cleaned_df['PERSONAL7'].apply(lambda x: 1 if x.find('Lost my job') >= 0 else 0) 
    cleaned_df['PERSONAL7_LOSS_CHILDCARE'] = cleaned_df['PERSONAL7'].apply(lambda x: 1 if x.find('Loss of childcare') >= 0 else 0) 
    cleaned_df['PERSONAL7_GETTING_FOOD'] = cleaned_df['PERSONAL7'].apply(lambda x: 1 if x.find('Getting food') >= 0 else 0) 
    cleaned_df['PERSONAL7_GETTING_SUPPLIES'] = cleaned_df['PERSONAL7'].apply(lambda x: 1 if x.find('Getting supplies') >= 0 else 0)
    cleaned_df['PERSONAL7_GETTING_MEDICATIONS'] = cleaned_df['PERSONAL7'].apply(lambda x: 1 if x.find('Getting routine') >= 0 else 0) 
    cleaned_df['PERSONAL7_TRANSPORTATION'] =  cleaned_df['PERSONAL7'].apply(lambda x: 1 if x.find('Transportation') >= 0 else 0)   
    cleaned_df['PERSONAL7_ACCESS_HEALTHCARE'] = cleaned_df['PERSONAL7'].apply(lambda x: 1 if x.find('Accessing healthcare') >= 0 else 0) 
    cleaned_df['PERSONAL7_NONE'] = cleaned_df['PERSONAL7'].apply(lambda x: 1 if x.find('None of the above') >= 0 else 0) 
    cleaned_df['PERSONAL7_OTHER'] = cleaned_df['PERSONAL7'].apply(lambda x: 1 if x.find('Other,') >= 0 else 0) 
    cleaned_df.drop('PERSONAL7', axis = 1, inplace=True)

if 'MEDIA3' in cleaned_df.columns:      
    cleaned_df['MEDIA3_TV'] = cleaned_df['MEDIA3'].apply(lambda x: 1 if x.find('TV') >= 0 else 0) 
    cleaned_df['MEDIA3_SOCIAL_MEDIA'] = cleaned_df['MEDIA3'].apply(lambda x: 1 if x.find('Social media') >= 0 else 0) 
    cleaned_df['MEDIA3_NEWSPAPER'] = cleaned_df['MEDIA3'].apply(lambda x: 1 if x.find('Newspapers') >= 0 else 0) 
    cleaned_df['MEDIA3_MAGAZINE'] = cleaned_df['MEDIA3'].apply(lambda x: 1 if x.find('Magazines') >= 0 else 0) 
    cleaned_df['MEDIA3_BLOG'] = cleaned_df['MEDIA3'].apply(lambda x: 1 if x.find('Blogs and podcasts') >= 0 else 0)
    cleaned_df['MEDIA3_GOV_WEBSITE'] = cleaned_df['MEDIA3'].apply(lambda x: 1 if x.find('government websites') >= 0 else 0) 
    cleaned_df['MEDIA3_HEALTH_WEBSITE'] =  cleaned_df['MEDIA3'].apply(lambda x: 1 if x.find('health department websites') >= 0 else 0)   
    cleaned_df['MEDIA3_FRIENDS_FAMILY'] = cleaned_df['MEDIA3'].apply(lambda x: 1 if x.find('Friends or family members') >= 0 else 0) 
    cleaned_df['MEDIA3_HEALTH_PRACTITIONERS'] = cleaned_df['MEDIA3'].apply(lambda x: 1 if x.find('Health practitioners') >= 0 else 0) 
    cleaned_df['MEDIA3_HEALTH_RESEARCHER'] = cleaned_df['MEDIA3'].apply(lambda x: 1 if x.find('Health researchers') >= 0 else 0) 
    cleaned_df['MEDIA3_SCIEN_ARTICLES'] = cleaned_df['MEDIA3'].apply(lambda x: 1 if x.find('Scientific articles') >= 0 else 0) 
    cleaned_df['MEDIA3_OTHER'] = cleaned_df['MEDIA3'].apply(lambda x: 1 if x.find('Other,') >= 0 else 0)
    cleaned_df.drop('MEDIA3', axis = 1, inplace=True)  
    
if 'MEDIA4' in cleaned_df.columns:      
    cleaned_df['MEDIA4_TV'] = cleaned_df['MEDIA4'].apply(lambda x: 1 if x.find('TV') >= 0 else 0) 
    cleaned_df['MEDIA4_SOCIAL_MEDIA'] = cleaned_df['MEDIA4'].apply(lambda x: 1 if x.find('Social media') >= 0 else 0) 
    cleaned_df['MEDIA4_NEWSPAPER'] = cleaned_df['MEDIA4'].apply(lambda x: 1 if x.find('Newspapers') >= 0 else 0) 
    cleaned_df['MEDIA4_MAGAZINE'] = cleaned_df['MEDIA4'].apply(lambda x: 1 if x.find('Magazines') >= 0 else 0) 
    cleaned_df['MEDIA4_BLOG'] = cleaned_df['MEDIA4'].apply(lambda x: 1 if x.find('Blogs and podcasts') >= 0 else 0)
    cleaned_df['MEDIA4_GOV_WEBSITE'] = cleaned_df['MEDIA4'].apply(lambda x: 1 if x.find('government websites') >= 0 else 0) 
    cleaned_df['MEDIA4_HEALTH_WEBSITE'] =  cleaned_df['MEDIA4'].apply(lambda x: 1 if x.find('health department websites') >= 0 else 0)   
    cleaned_df['MEDIA4_FRIENDS_FAMILY'] = cleaned_df['MEDIA4'].apply(lambda x: 1 if x.find('Friends or family members') >= 0 else 0) 
    cleaned_df['MEDIA4_HEALTH_PRACTITIONERS'] = cleaned_df['MEDIA4'].apply(lambda x: 1 if x.find('Health practitioners') >= 0 else 0) 
    cleaned_df['MEDIA4_HEALTH_RESEARCHER'] = cleaned_df['MEDIA4'].apply(lambda x: 1 if x.find('Health researchers') >= 0 else 0) 
    cleaned_df['MEDIA4_SCIEN_ARTICLES'] = cleaned_df['MEDIA4'].apply(lambda x: 1 if x.find('Scientific articles') >= 0 else 0) 
    cleaned_df['MEDIA4_OTHER'] = cleaned_df['MEDIA4'].apply(lambda x: 1 if x.find('Other,') >= 0 else 0)
    cleaned_df.drop('MEDIA4', axis = 1, inplace=True)  
    
if 'MEDIA8' in cleaned_df.columns:      
    cleaned_df['MEDIA8_DIABETES'] = cleaned_df['MEDIA8'].apply(lambda x: 1 if x.find('Diabetes') >= 0 else 0) 
    cleaned_df['MEDIA8_HEART_DISEASE'] = cleaned_df['MEDIA8'].apply(lambda x: 1 if x.find('Heart disease') >= 0 else 0) 
    cleaned_df['MEDIA8_CANCER'] = cleaned_df['MEDIA8'].apply(lambda x: 1 if x.find('Cancer') >= 0 else 0) 
    cleaned_df['MEDIA8_HIV'] = cleaned_df['MEDIA8'].apply(lambda x: 1 if x.find('HIV') >= 0 else 0) 
    cleaned_df['MEDIA8_ASTHMA'] = cleaned_df['MEDIA8'].apply(lambda x: 1 if x.find('Asthma') >= 0 else 0)
    cleaned_df['MEDIA8_LUNG_DISEASE'] = cleaned_df['MEDIA8'].apply(lambda x: 1 if x.find('lung Disease') >= 0 else 0)      
    cleaned_df['MEDIA8_OTHER'] = cleaned_df['MEDIA8'].apply(lambda x: 1 if x.find('Other') >= 0 else 0)
    cleaned_df['MEDIA8_NONE'] = cleaned_df['MEDIA8'].apply(lambda x: 1 if x.find('None') >= 0 else 0)
    cleaned_df['MEDIA8_HYPERTENSION'] = cleaned_df['MEDIA8'].apply(lambda x: 1 if x.find('Hypertension') >= 0 else 0)    
    cleaned_df.drop('MEDIA8', axis = 1, inplace=True)  
    
if 'MEDIA16' in cleaned_df.columns:  
    cleaned_df['MEDIA16_SOUP_KITCHEN'] = cleaned_df['MEDIA16'].apply(lambda x: 1 if x.find('soup kitchen') >= 0 else 0) 
    cleaned_df['MEDIA16_SCHOOL'] = cleaned_df['MEDIA16'].apply(lambda x: 1 if x.find('school district') >= 0 else 0) 
    cleaned_df['MEDIA16_FOOD_STAMPS'] = cleaned_df['MEDIA16'].apply(lambda x: 1 if x.find('food stamps') >= 0 else 0) 
    cleaned_df['MEDIA16_COMMUNITY'] = cleaned_df['MEDIA16'].apply(lambda x: 1 if x.find('community food') >= 0 else 0) 
    cleaned_df['MEDIA16_FOOD_ASSIS'] = cleaned_df['MEDIA16'].apply(lambda x: 1 if x.find('food assistance') >= 0 else 0)
    cleaned_df['MEDIA16_DONT_KNOW'] = cleaned_df['MEDIA16'].apply(lambda x: 1 if x.find("Don't know") >= 0 else 0)  
    cleaned_df.drop('MEDIA16', axis = 1, inplace=True)
    
if 'RUMORS1' in cleaned_df.columns:
    cleaned_df['RUMORS1_ACCIDENTLY_ESCAPED'] = cleaned_df['RUMORS1'].apply(lambda x: 1 if x.find('accidently escaped') >= 0 else 0) 
    cleaned_df['RUMORS1_PURPOSEFULLY_LEAKED'] = cleaned_df['RUMORS1'].apply(lambda x: 1 if x.find('purposefully leaked') >= 0 else 0) 
    cleaned_df['RUMORS1_USARMY_BROUGHT'] = cleaned_df['RUMORS1'].apply(lambda x: 1 if x.find('The US army brought') >= 0 else 0) 
    cleaned_df['RUMORS1_BAT_TO_HUMANS'] = cleaned_df['RUMORS1'].apply(lambda x: 1 if x.find('from bat to humans') >= 0 else 0)
    cleaned_df['RUMORS1_BAT_ANIMAL_HUMANS'] = cleaned_df['RUMORS1'].apply(lambda x: 1 if x.find('from bats to another animal') >= 0 else 0) 
    cleaned_df['RUMORS1_WET_MARKET'] = cleaned_df['RUMORS1'].apply(lambda x: 1 if x.find('It was introduced in a') >= 0 else 0) 
    cleaned_df['RUMORS1_OTHER'] = cleaned_df['RUMORS1'].apply(lambda x: 1 if x.find('Other,') >= 0 else 0) 
    cleaned_df.drop('RUMORS1', axis = 1, inplace=True)

listOfStudents = ['Student', 'student', 'am a student', 'I am a full time student.', 'Unemployed  student']    
listOfUnemployed = ['Unemployed', 'Unemployed, unable to work', 'Unemployed, looking for for', 
                'unemployed contractor', 'unemployed', 'Unemployed/Disabled', 
                 'Unemployed, unable to work', 'Unemployed  student']

listOfRetired = ['Retired', 'retired', 'retired; some house sitting', 'Retired postal employee',
                'retired loan officer', 'retired disabled', 'Retired librarian', 
                 'Retired. Fulltime Caregiver for husband.', 'retired health care',
                'retired educator', 'retired and happy', 'I am retired.', 'Retired counselor',
                'RETIRED  PREVIOUS WAS AN ACCOUNTANT', 'retired teacher', 'Retired Dental Hygienist',
                'retired administrative assistant', 'fully retired', 'Retired RN',
                'Retired. Hoping to work part time.', 'Retired Accountant', 
                 'Retired veterinary technician', 'retired chemist']

listOfJobs = ["Manufacturing ","Account","Childcare ","Transportation ","paralegal ",
              "Rehab assistant ","Mechanic ","public health","education ","driving ","Analysist",
              "Transportation ","sales","manager ","Director ","Caregiver ",
              "Customer service representative","Healthcare ","Media Blogger ","clergy",
              "Preschool teacher ","freelancer","Hr","Office","Temporary Labor","Courier ", 
              "It Director","Blogger","Graphic designer/photographer ","Self employed","Theme Parks ",
              "IT","Nurse","Business owner","salesman","Technical","Deli associate","Sales",
              "Finsnce","dont work","Office staff at a school","Bar manager","Farmer","Geological ",
              "worker","i have an online retail store","Decorator ","Management","Supervisor ",
              "Director ","Finance","manager ","manager","Business ","Engineer ","it","Teacher",
              "Retail merchandiser ","Beautician","Sakes manager","clerk","Nutritionist ","Director",
              "Teacher","Information technology manager","Business man","Journalist",
              "Customer service specialist","Data analyst ","IT professional","housekeeper","it",
              "Financial services","Team lead","Social worker","Self Employed","Health ","Daycare",
              "Retail ","Physical therapy","Manager","Plumber ","Retail ","Barista",
              "MANUFACTURA","Engineering","Office coordinator ","Seller","Bank","Accounting",
              "Cannabis trimmer","it programmer","Manufacturing","It officer","Construction",
              "Farmer","Manager","engineer","IT","Computer ","Legal","Retail ",
              "government transcriptionist","Engineering","Healthcare","Accounting","Manufacturing",
              "Retail Associate ","Manager","Information technology officer","Art museum worker ",
              "Cashier ","Computer engineer","Marketing ","Stay at home mom","Constrution","Banker",
              "Legal assistant ","Educator","Homemaker","Banks ","Banking ","Manager","Doctor",
              "Artist ","Typist ","Banking","Engineer","Cashier","Plumber","nurise","Security",
              "Financial manager","Sales and Marketing Manager at a retail store.","Warehousing ",
              "It/communications ","Computer analyst","nursing home","Chef","Retail","Housekeeper",
              "Homemaker ","owner of a small business ","Information technology","Telecommunications",
              "Ecommerce seller of handmade jewelry","Teacher ","Financial analyst ","An accountant",
              "Military personnel","sales clerk","Teaching ","Transportation ","Marketing ",
              "Grocery ","IT","Information technologist","eCommerce Associate","Accounting",
              "Information technology","Medical lab Scientist","Community organizer ",
              "A financial expert","Dating service Director","Nurse","constructor","Accountanting",
              "Healthcare","Information Technology ","Health Aide","Security ",
              "computer software and hardware and development","Flea market","Construction",
              "Consultation ","Banker","Educator ","computer software","Software developer ",
              "Plumber ","Painter","Banker","Engineer","IT tecnology","IT","Information technology",
              "Banking and finance ","Typist","engineer","Software Engineer","Trainer ","Direto",
              "Manufacturer","finance","Financial services ","Diretor ","Doctore",
              "alchohol distribution","Accountant ","Purchasing director","Information technology",
              "Manager","Director of IT","Officer in charge","Automotive","Bank branch manager",
              "Director","Financial institution ","Technology","Finance ","Prep Cook","Developer ",
              "Accountant ","Manager of tech","Software developer","Financial advisor","IT Manager ",
              "Xray technologist","Insurance","IT director","teacher","Insurance",
              "Department of defense","Academian ","Telecommunication.","Construction",
              "Agriculture ","Tailoring ","Clerical Staff","Information technology / IT",
              "Information Technology ","Retai ","ADMIN","Construction","IT","Permanently disabled ",
              "Medical Transcriptionist","Sales","Nurse aide","Self-Employed","Food Manager",
              "Civil servant","Caregiver","IT MANAGER","Data analyst","Disabled, was a chemist.",
              "Analyst","ceo","Cheif information officer ","IT Instructor","Sales","Quality control ",
              "Retired","IT","Any work that involves money ","Financial services","manager",
              "retail marketing ","Arts and equestrian and recreation events ","Manager",
              "Insurance ","quality assurance inspection technician","Graphic Design",
              "Student transportation","Healthcare assistant","developer","Legal assistant","ASPCA",
              "Nurse","Accounting","Admin Assistant","Retail Associate","Manager","cashier",
              "Consulting","HEALTHCARE","Education industry ","dishwasher","Financial Analyst ",              
              "librarian","Receptionist","caregiver","Realtor","Nurse ","Accountant",
              "program manager","pbx operator","staffing","Banker","Network Engineer","Retail",
              "writer","Business","Information technology ","Dumcuter","CPA","transcription",
              "Employee","Engineer","Construction","Distribution","Tax Analyst",
              "Information technology","Information technologist ","Engineer ","Employed","Cook",
              "Senior Manager","Teacher","IT","Engineer","Information Technologist","Caretakwr",
              "construction","New Custom Home Painter","Retired. Fulltime Caregiver for husband.",
              "retired health care","retired","Banking and finance","Information and technology ",             
              "Financial analysts manager ","Healthcare","IT/Computer services","Insurance",
              "Real estate manager","Automotive ","Health worker","It",
              " work in the HR department","Education","sales","IT expert","student",
              "Pharmacy technician ","Business owner","Information technology","Insurance",
              "Manufacturer","Football player ","general contractor","homemaker","management ",
              "Real Estate","healthcare","Information Technology","consultant","Educator",
              "Office assistant","Plumber ","Surgeon ","healthcare","teacher","MACHINING","Designer",
              "Retail sales","school cafe worker","Student","Student ","homemaker","Social services",
              "Information Technology","Construction ","Farmer ","Cashier","i work",
              "Director Loyalty ","Accounting ","education","Teacher","Childcare ","Pet care",
              "Director","I.T Manager","Technician","manager of a private company",
              "I work in the Information Technology.","Warehouse Clerk","Customer Service Rep",
              "IT","Management consultant","Retired","Senior Computer Systems Engineer","Advise",
              "Market research","Cook","Healthcare ","Order Entry(Temporary/Seasonal).","cashier",
              "Healthcare ","Civil service ","Landscaping helper","student","Healthcare","driver",
              "merchandiser","freelancer","Waretired private t duty hospice care provider",
              "IT Director","Barista","The best experience ","Educator ","Nurse","Dealer",
              "Geological ","Director","Manager","Accountant","Construction","Data analysis ",
              "Telecommunications ","saller","assistant","Manager at IT company","farmer",
              "A writer.","Financial Advisor","Retail","Manager","Programmer","Financial officer",
              "Information technology ","Telecommunications director ","Warehouse","Technologist",
              "Teacher","IT manager","banker","Constructor ","Blogger and advertiser","Engineer ",
              "Retail associate and student","Database Analyst","medical  administrator",
              "construction/carpenter","Information technologist","Typist","Engineer",
              "Banking and finance ","Pampered chef Rep","2","Manager","retired teacher",
              "Social Worker","information tehnology","IT ","Retired Dental Hygienist ","Banker",
              "IT","Electrical enginner","HOMEMAKER","freelance writer","IT  analyst ","Consultant",
              "nurse assistant","Hotel Manager","IT Professional","Inventory Management","Farmer",
              "Teacher","IT","Banking and financial services ","social worker","Good great ",
              "Manager","Manager of a company","Banker","employed full time","Farmer","Nurse",
              "Realtor ","General construction","BANKING FINACES","Administrator","Encanador",
              "Factory worker","Clerical ","Retail","Typist","Freelance","adasdsadsa",
              "InformationTechnology","CEO","management ","3","IT","Administrative ",
              "Accountant","Lawery","Doctor","Restaurant ","Administrator","healthcare aide",
              "Financial advisor","Engineer","independent contractor","plumber","carpenter",
              "Tech manufacturer","Accountant ","Banking","Finances","Financial Director",
              "Business analyst","Paralegal ","Financial director","Scientist ","Education",
              "Docter","Education","Hair stylist ","Executive","Financial controller ",
              "Administrator","Manager","Businessman","Nurse ","Banking and finance",
              "Parish administrator ","Financial advisor","Banker","retail/whole sale",
              "I like","support specialist","IT Expert","Computer engineer","Consultant",
              "Accountant","PHYSICIAN","Healthcare ","Information technologist",
              "Managing director","microbiologist","Medical Doctor","Sweet compsny ","dentist",
              "Financial specialist","Farmer","GIG Delivery for Postmates","Business administrator",
              "HVAC ","Manager","Civil Engineer","Health care","Tecnology","Healthcare","Doctor",
              "Teacher","Middle manager","Insurance","Designer ","semi retired retail","Associate ",
              "HR","Artist ","office management ","Retail ","Farmer","Arts",
              "Customer Service Representative","Construction","Healthcare","Contractor",
              "Director for a boys and girls club ","0","Manager ","manager","Paint","IT",
              "manager","Package handler ","Substitute teacher","Art","Customer service ",
              "Freelance Author","Child Care","Good","Foreman ","Receptionist ","Engineer",
              "Data entry clerk ","Photographer ","Sales","healthcare ","Education","Car detail",
              "Journalism","Self emoyeed","Counselor ","Transportation ","Home maker","Walmart ",
              "Tefinish furniture","Information technology","I work at Mcdonalds fast food ",
              "Management","Martial arts trainer","Retail","Information technology","manager",
              "Emwd","Retired veterinary technician","Banker","Education","project manager",
              "Creative designer ","information technology","hr mgr","Crocheter","Plumber","Banker ",
              "Entrepreneur in the mental health space","Consulting","IT manager","Secretary",
              "Consultant","Construction","Photography","Tech","plumber","Cleaner",
              "Proffesional business ","Hacker","Famous rapper ","Nurse aide ","Retail ",
              "Dogwalker","Health care","manager","Encoder","Information technology","Consultant ",
              "Education","Customer service ","IT","food service","Information  technology",
              "Data specialist ","IT","IT MANAGER","Food service Worker ","information technology",
              "Education","manufacture","That could be very good and information technology.",
              "medical assistant","Food server","Home health","Estate clearances","It Consult",
              "Interior decorator ","Tech","Restaurant Manager","manufacturing","Intern",
              "Receptionist","sales manager","Engineering","retail sales ","Construction",
              "State/government","Banker ","General contractor ","Banker","Chuef technology officer",
              "Marketer","Information Technologist consultant","Electrician","Engineering ","IT",
              "Manager ","Chef","Real estate ","Engineer","Security Officer","Information Technology",
              "Information technology","IT","Freelance ","Gnome","Banker","Part-time worker ",
              "Sinticts","Information Technology ","Plumer typist","Software engineer","Advertising",
              "Financial institution ","Sales","IT professional ","Teacher","Economist",
              "i working banking and financilal project","Programmer","Banking/finance ",
              "IT related","Construction","House cleaning","Mechanic ","Teacher","private job",
              "Chemist","Business man","Make eoeneoemekem","Health care ","Management",
              "medical records clerk","Finance","clerk","Information Technology","Chef","Teacher",
              "Finace","Auto detailing","Director","Business services business owner","entrey level",
              "IT specialist","stay at home parent","Homemaker ","teacher..4th grader teacher",
              "Music Engineer","I. T","Proffesional business ","Accountant","manager",
              "times plumber","construction","Plumber aSsiant ","Disabled ","Accountant ",
              "Procurement Analyst","Modeling ","Administrator","Financial advisor","Finance",
              "Information technology","Doctor","real estate","Self employed","graphic design",
              "Director","Programmer","Information technology","information technology ",
              "Accounting","Kitchen worker","Plumer","Insurance "]
 
listOfAll= listOfJobs + listOfUnemployed + listOfRetired + listOfStudents

if 'DEMOGRAPHIC1' in cleaned_df.columns:
    
    cleaned_df['DEMOGRAPHIC1_EMPLOYED']=cleaned_df['DEMOGRAPHIC1'].apply(lambda x: 1 if x in listOfJobs else 0) 
    cleaned_df['DEMOGRAPHIC1_UNEMPLOYED']=cleaned_df['DEMOGRAPHIC1'].apply(lambda x: 1 if x in listOfUnemployed else 0) 
    cleaned_df['DEMOGRAPHIC1_RETIRED']=cleaned_df['DEMOGRAPHIC1'].apply(lambda x: 1 if x in listOfRetired else 0)
    cleaned_df['DEMOGRAPHIC1_STUDENTS']=cleaned_df['DEMOGRAPHIC1'].apply(lambda x: 1 if x in listOfStudents else 0) 
    cleaned_df['DEMOGRAPHIC1_OTHER']=cleaned_df['DEMOGRAPHIC1'].apply(lambda x: 0 if x in listOfAll else 1) 
    cleaned_df.drop('DEMOGRAPHIC1', axis = 1, inplace=True)    


In [21]:
 #Check for missing values, fill nan
for (columnName, columnData) in cleaned_df.iteritems(): 
    missingCount = columnData.isnull().sum() 
    percentage = (missingCount/totalRows)*100 
    #print(percentage)
    if missingCount > 0:        
        print("Missing Values in column : ", columnName, ", missing rows: ", 
            missingCount, ", missing percentage: ",  percentage, '%')

Missing Values in column :  HR11 , missing rows:  2 , missing percentage:  0.16515276630883566 %
Missing Values in column :  COVID_HR1 , missing rows:  485 , missing percentage:  40.04954582989265 %
Missing Values in column :  COVID_HR2 , missing rows:  404 , missing percentage:  33.360858794384804 %
Missing Values in column :  IDEOLOGY4 , missing rows:  323 , missing percentage:  26.67217175887696 %


In [22]:
 #Fill missing values
cleaned_df.fillna('NA', inplace=True)
try:
    cleaned_df.to_csv("cleaned_df.csv") 
except PermissionError as prr:
    print("Cannot save file if you have it open in Windows ")

